In [22]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

url = 'https://mbasic.facebook.com'
driver = webdriver.Chrome()
driver.get(url)

email_input = driver.find_element_by_name("email")
password_input = driver.find_element_by_name("pass")

email_input.send_keys("ching6544239@yahoo.com")
password_input.send_keys("z4901488")

login_button = driver.find_element_by_name("login")
login_button.click()

later_button = driver.find_element_by_link_text("稍後再說")
later_button.click()

search_box = driver.find_element_by_name("query")
search_box.send_keys("詹記麻辣鍋")
search_box.submit()
time.sleep(5)

later_button = driver.find_element_by_link_text("更多")
later_button.click()

later_button = driver.find_element_by_link_text("貼文")
later_button.click()


    
    
    
#     # 遍历每个文章
#     for article in articles:
#         # 提取 header 内容
#         header = article.find_element_by_tag_name("header").text

#         # 使用 XPath 找到 header 後面的第一個以 "c" 開頭的 div
#         div_c = article.find_element_by_xpath('.//header/following-sibling::div[starts-with(@class, "c")]')

# #         div_cm = article.find_element_by_css_selector('div.cm')
#         p_elements = relative_div.find_elements_by_tag_name('p')

#         content_list = []
#         for p_element in p_elements:
#             content_list.append(p_element.text)

#         # 保存为JSON格式
#         result = {"header": header, "content": content_list}
#         print(json.dumps(result, ensure_ascii=False))

    # 尝试点击"查看更多結果"链接
    try:
        more_results_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "search/posts/") and contains(span, "查看更多結果")]'))
            )
        more_results_link.click()
        
        WebDriverWait(driver, 10).until(
            EC.staleness_of(articles[-1])  # 等待上一頁的最後一個 article 消失，表示新頁面已經加載完成
        )

        time.sleep(3)
    except Exception as e:
        print("No more results link found.")
        break

# driver.quit()


IndentationError: unexpected indent (<ipython-input-22-e4230e65c52e>, line 43)

In [23]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

url = 'https://mbasic.facebook.com'
driver = webdriver.Chrome()
driver.get(url)

email_input = driver.find_element_by_name("email")
password_input = driver.find_element_by_name("pass")

email_input.send_keys("ching6544239@yahoo.com")
password_input.send_keys("z4901488")

login_button = driver.find_element_by_name("login")
login_button.click()

later_button = driver.find_element_by_link_text("稍後再說")
later_button.click()

search_box = driver.find_element_by_name("query")
search_box.send_keys("詹記麻辣鍋")
search_box.submit()
time.sleep(5)

later_button = driver.find_element_by_link_text("更多")
later_button.click()

later_button = driver.find_element_by_link_text("貼文")
later_button.click()

# 循环爬取前5页的搜索结果
for page in range(10):
    try:
        # 等待所有 article 元素加载完成
        articles = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'article'))
        )

        # 遍历每个文章
        for article in articles:
            # 提取 header 內容
            header = article.find_element_by_tag_name("header").text

            # 使用 XPath 找到 header 後面的第一個以 "c" 開頭的 div
            try:
                div_c = article.find_element_by_xpath('.//header/following-sibling::div[starts-with(@class, "c")]')
                p_elements = div_c.find_elements_by_tag_name('p')

                content_list = []
                for p_element in p_elements:
                    content_list.append(p_element.text)

                # 保存為 JSON 格式
                result = {"header": header, "content": content_list}
                print(json.dumps(result, ensure_ascii=False))
            except NoSuchElementException:
                print("No div with class 'c' found.")

        # 尝试点击"查看更多結果"链接
        more_results_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//a[contains(@href, "search/posts/") and contains(span, "查看更多結果")]'))
        )
        more_results_link.click()

        WebDriverWait(driver, 10).until(
            EC.staleness_of(articles[-1])  # 等待上一頁的最後一個 article 消失，表示新頁面已經加載完成
        )

    except TimeoutException:
        print("TimeoutException: Timed out waiting for page to load.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        break


{"header": "暉哥 Faigor──在詹記麻辣火鍋 敦南店。", "content": ["#暉哥吃火鍋\n吃到鼎鼎大名的#詹記\n麻辣湯底可以叫到中辣🌶️\n滿足了重口味的我😋\n湯的基底即便沒有大骨🦴\n中藥的香味已經很濃郁😮‍💨\n適合喜歡吃麻辣鍋的你🌿\n價錢不便宜💸\n但肉品品質不錯🥩\n#暉哥吃播"]}
{"header": "小圓吃喝玩樂筆記──在詹記麻辣火鍋 敦南店。", "content": ["有名的詹記麻辣鍋，", "激推👍鴨血，\n吃起來真的很嫩很滑，\n屌打小編之前吃過的，\n但不是吃到吐（要$）\n小鍋只能再加點一份（可以點半份，2個半份也可）", "ㄧ定是小編壓力大，\nㄧ進門有種醫院的fu，就醫style，⋯⋯"]}
{"header": "Spyci 吃飯趣──在詹記麻辣火鍋 敦南店。", "content": ["6/14中午吃了詹記麻辣火鍋\n好有創意的餐廳😂\n進門就是一個電梯，服務員說要到幾樓都可以，或是按開門，結果不管按幾樓到達的目的地都在1樓😂\n麻辣鍋湯底味道就一般般，沒有好或不好，但是肉質很好👍\n算是一個不錯的體驗"]}
{"header": "林聖欣──在詹記麻辣火鍋 敦南店。", "content": ["台北-詹記麻辣鍋\n台北老字號名店，真的名不虛傳！\n要等上1個多小時才輪得到來吃！\n會一直想在吃的麻辣鍋，有時間我一定要再訪！\n#詹記麻辣火鍋\n#哥哥吃台北\n#台北美食\n#台中美食 #苗栗美食 #桃園美食 #基隆美食 #新北美食 #新竹美食 #彰化美食 #嘉義美食 #南投美食 #雲林美食 #台南美食 #高雄美食 #屏東美食 #台東美食 #花蓮美食 #宜蘭美食 #金門美食 #澎湖美食 #馬祖美食 #台灣美食"]}
{"header": "昆娜的微旅食光──在詹記麻辣火鍋 敦南店。", "content": ["詹記麻辣火鍋 敦南店", "終於在10/21重新開幕", "剛好天氣也冷了", "那只好來跟風一下", "嗯"]}
{"header": "吃貨專門店──在詹記麻辣火鍋 敦南店。", "content": ["0630\n突然好想吃麻辣鍋🥹", "雖然這幾天已經熱到不能再熱，但不知為何還是很想吃麻辣鍋，突然想起之前吃的詹記。", "雖然價格不低，畢竟一盤牛肉要好幾百，但在乾淨舒適的冷

{"header": "余瑄穎覺得開心──和 Angel Lin 及葉姵均，在貿商社區。", "content": ["2021年最後一聚", "#詹記麻辣鍋"]}
{"header": "阿德作筆記覺得可愛──在詹記麻辣火鍋-新莊總店。", "content": ["[一切都好久不見，姐姐、詹記、夜遊、麻辣鍋]\n這一週真的非常漫長，即使昨日先短暫度過，\n但今日依舊又有好多迫在眉睫的事情要處理，\n只能說陀螺人生就是轉個不停，\n還有今天還要搬座位，幸好就在對面，\n搬起位子來相當輕鬆，而且我其實也沒啥東西！\n本想著今天是否能早些下班前去赴約，\n殊不知搞一搞也是到了八九點，\n也好險是跟半年不見的姐接相約於老巢新莊，\n才得以不會讓她無所適從！"]}
{"header": "WalkerLand 窩客島", "content": ["又有藉口吃 #詹記 了！\n88元牛五花、無限量加點~超佛~~ #喵小編", "詹記麻辣火鍋-新莊總店\n(一樓有最新防疫資訊、最夯外帶優惠)"]}
{"header": "詹記麻辣火鍋 敦南店", "content": ["詹記外帶一人鴛鴦鍋\n4月2日（四）\n明！天！上！市！囉！", "一共5種選擇：\n1. 美國翼板牛鴛鴦鍋 $550\n2. 美國牛五花鴛鴦鍋 $500\n3. 台灣松阪豬鴛鴦鍋 $500\n4. 台灣梅花豬鴛鴦鍋 $450\n5. 台灣培根豬鴛鴦鍋 $450"]}
{"header": "小精翎 唐明翎在吃晚餐──於詹記麻辣火鍋-新莊總店。", "content": ["終於來吃想吃很久的詹記新莊總店\n之前吃過N次南京店\n搬家後就沒再吃過\n剛好逮到機會可以來總店！\n我只能說訂位非常難訂，現場候位也頗久的，請一定要提早去拿號碼牌，手機網頁可以直接看候位進度\n我們加了兩次辣油還是三次\n不知道是整桌舌頭都壞了還是怎樣都不覺得辣🤣\n鴨血一如既往的好吃入味\n不知道是不是很多人反應太鹹，這次吃我並不覺得鹹，店員也有貼心說如果覺得鹹他們可以過來加高湯，應該是調整過了吧？\n要給詹記評語的話我會說-⋯⋯"]}
{"header": "Angel安啾──在詹記麻辣火鍋 敦南店。", "content": ["聽說……我今年的母親節大餐就是【詹記麻辣鍋】\n很久沒來店裡吃\n還是好好吃❤❤❤", "Adam

{"header": "詹記麻辣火鍋 敦南店", "content": ["** 詹記敦南店 < 社交公關部 > 撰稿人 詹紅龍 **\n‍‍‍‍‍‍ ‍‍\n- 第二章 甜點篇 -\n‍‍‍‍‍‍ ‍‍\n看著凍僵成堆的詹記員工，米奇店長站在冷凍庫前眉頭深鎖，二代目拉著他的外套衣角，眼尾泛淚。\n「你真的要進去嗎？」\n「雪寶要包完，客人需要雪寶。」\n「我需要你。」\n「養紅龍太花錢了，包裝只能自己做。」\n‍‍‍‍‍‍ ‍‍⋯⋯"]}
{"header": "詹記麻辣火鍋 敦南店", "content": ["你各位啊～手刀搶起來～", "沒看過鍋子三個月沒工作～\n無聊到跳樓嗎？", "開\n放\n內\n用\n了⋯⋯"]}
{"header": "吳小婷", "content": ["好久沒來現場吃詹記麻辣鍋\n託高雄好友北上的福有了這個幸福午餐之約\n真的是太美味了啦😋😋😋", "照貫例跟我出門吃飯一坐下\n我就會開始發防胖法寶（吸油阻醣）一定要\n飆瘦計畫執行中也能開心享用美食\n說這不是最幸福的瘦身方式是什麼？", "11月忙完新品收單\n天天都要開心的過\n年底就是有好多的約會好多的飯局\n一路都要吃的開心又安心", "這兩天台北一樣像夏天！\n夏天一直不跟我分手，我比較喜歡秋天說！", "#想防胖想瘦身想變美的都歡迎來問我喔"]}
{"header": "Ray Yiu > 台北新北租屋🌞🌞🌞（免仲介費）", "content": ["感謝管理員審核\n格局方正採光超好,文湖線和平敦南生活圈，適合家庭或朋友一同居住\n【房東自租】\n3房2廳1衛\n【地點】\n大安區和平東路及敦化南路口,和平東路三段\n【房型】\n公寓大樓\n一層兩戶\n位於4樓⋯⋯"]}
{"header": "詹記麻辣火鍋 敦南店", "content": ["㊗️🎉狂賀🎉㊗️ 詹記大吉烏梅啤酒🍺\n銷售3個月突破5000支！🏆🏆🏆🏆🤚👋🤚👋👏\n又濕又冷的天氣🌧️吃麻辣鍋配烏梅啤酒最棒了！\n鍋裡咻滾棍，肚子也翻滾滾\n再喝一口酸酸甜甜、去油解膩、喝了會嗨的烏梅啤酒\n啊嘶～～～～👀👄🍲🍻🧠💗💣💥\n・\n💫這次詹記又跟神級人物合作了！✨\n恭迎～唯美派插畫始祖✨ 平凡＆陳淑芬✨\n以及台灣唯一擁有8塊腹肌的設計師廖小子⋯⋯"]}
{"header": "瑪姫幸福過日子",

NameError: name 'TimeoutException' is not defined

In [5]:
import requests
from bs4 import BeautifulSoup

In [7]:
# 发送HTTP请求，获取网页内容
response = requests.get("https://www.managertoday.com.tw/articles/view/67130")
print(response.status_code)

# 检查响应状态
if response.status_code == 200:
    # 使用Beautiful Soup解析网页内容
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 找到目标 <div> 元素
    target_div = soup.find("div", class_="px-5 md:px-0")
    
    if target_div:
        # 找到 <main> 元素
        main_element = target_div.find("main")
        
        if main_element:
            # 遍历 <main> 元素内的所有文本内容
            extracted_text = ""
            for element in main_element.find_all(["p", "h2", "h3", "h4","strong"]):
                extracted_text += element.get_text() + "\n"

            # 输出提取的文本内容
            print(extracted_text)
        else:
            print("未找到 <main> 元素")
    else:
        print("未找到目标 <div> 元素")
else:
    print("无法访问网页或获取内容")


200
輝達（NVIDIA）執行長黃仁勳、超微（AMD）執行長蘇姿丰相繼來台，掀起 AI 熱潮！據 104 人力銀行徵才資料庫顯示，目前市場上開出逾 2.6 萬個 AI 相關工作機會。其中近六成集中於電子資訊、軟體、半導體業，近兩成來自一般製造業，而批發、零售、傳直銷業、法律、會計、顧問、研發及設計業也都有相關的職缺釋出，已見各產業同時需求 AI 人才。
AI 應用人才搶手！跨五大產業，企業徵才逾 2.6 萬人
蘇姿丰日前接受媒體聯訪時指出， AI 目前仍在拓荒的階段，但她十分看好接下來 5 – 10 年的發展， 超微會將 AI 當作首要的投資項目，其中硬體設備從 CPU（中央處理器）、GPU（繪圖處理器）、FPGA（現場可程式化邏輯閘陣列）和 ASIC（客製化 IC）都需要 AI 解決方案，未來 5 年內產值將成長至 1500 億美元。
AI 目前仍在拓荒的階段，但她十分看好接下來 5 – 10 年的發展，
104 人力銀行數據長李魁林觀察，企業對 AI 人才的需求，分為「用 AI」的工程研發相關、「作 AI」的應用解決相關，工作者只要嫻熟熱門的 AI 應用工具，都有機會切入 AI 領域。 進修 AI 相關技能，跳槽轉職、或原職含金量都能夠幫自己爭取至少一成以上的加薪幅度。 「懂 AI」則是熟玩 AI 應用產品、了解背後的運作原理和機制，更能夠接近產業的當紅核心，同時 AI 應用與整合力也將成為目前工作者的必備技能
進修 AI 相關技能，跳槽轉職、或原職含金量都能夠幫自己爭取至少一成以上的加薪幅度。
AI 人才最常運用哪些工具？
工作者常用到生成式 AI 工具，以最近熱門的 ChatGPT、Midjourney 以及 Stable Diffusion 為例，近三個月內企業徵才註明相關技能的職缺也明顯增加，要求 ChatGPT 技能的職缺有 177 個，要求 Midjourney 技能的職缺有 37 個，要求 Stable Diffusion 技能的職缺有 255 個。
職缺 AI 化的發展機會是？
在 AI 風潮新增的相關工作機會中，除了直接對應到研發工程師，也有不少是在生成式 AI 的發展下，傳統大宗職缺 AI 化：
AI 行銷企劃 (AI Marketing Specialist)
傳統職務：行銷企劃、網站行銷企劃。
AI 化之後：參與 AI 團隊合作，進行各

In [8]:
import re
import string

# 假设你已经提取的文本保存在 extracted_text 变量中
text = extracted_text

# 使用正则表达式删除 "資料來源" 及其后的内容
pattern = r'資料來源.*'
text_without_source = re.sub(pattern, '', text)

# 進行文本內容清理
cleaned_text = re.sub(r'[。：「（），\n」\s]+', '', text_without_source)
cleaned_text = cleaned_text.replace("的", "")

In [9]:
cleaned_text

'輝達NVIDIA執行長黃仁勳、超微AMD執行長蘇姿丰相繼來台掀起AI熱潮！據104人力銀行徵才資料庫顯示目前市場上開出逾2.6萬個AI相關工作機會其中近六成集中於電子資訊、軟體、半導體業近兩成來自一般製造業而批發、零售、傳直銷業、法律、會計、顧問、研發及設計業也都有相關職缺釋出已見各產業同時需求AI人才AI應用人才搶手！跨五大產業企業徵才逾2.6萬人蘇姿丰日前接受媒體聯訪時指出AI目前仍在拓荒階段但她十分看好接下來5–10年發展超微會將AI當作首要投資項目其中硬體設備從CPU中央處理器、GPU繪圖處理器、FPGA現場可程式化邏輯閘陣列和ASIC客製化IC都需要AI解決方案未來5年內產值將成長至1500億美元AI目前仍在拓荒階段但她十分看好接下來5–10年發展104人力銀行數據長李魁林觀察企業對AI人才需求分為用AI工程研發相關、作AI應用解決相關工作者只要嫻熟熱門AI應用工具都有機會切入AI領域進修AI相關技能跳槽轉職、或原職含金量都能夠幫自己爭取至少一成以上加薪幅度懂AI則是熟玩AI應用產品、了解背後運作原理和機制更能夠接近產業當紅核心同時AI應用與整合力也將成為目前工作者必備技能進修AI相關技能跳槽轉職、或原職含金量都能夠幫自己爭取至少一成以上加薪幅度AI人才最常運用哪些工具？工作者常用到生成式AI工具以最近熱門ChatGPT、Midjourney以及StableDiffusion為例近三個月內企業徵才註明相關技能職缺也明顯增加要求ChatGPT技能職缺有177個要求Midjourney技能職缺有37個要求StableDiffusion技能職缺有255個職缺AI化發展機會是？在AI風潮新增相關工作機會中除了直接對應到研發工程師也有不少是在生成式AI發展下傳統大宗職缺AI化AI行銷企劃(AIMarketingSpecialist)傳統職務行銷企劃、網站行銷企劃AI化之後參與AI團隊合作進行各種實體與線上活動規劃與執行對外推廣品牌利用AI技術優化並推動B2B國際媒體行銷、社群媒體經營、以達到最佳轉換效果使用AI智庫嵌入公司網站及產品資訊推動國際行銷提供最佳客戶體驗結合AI與數位行銷工具SEO、SEM、Google廣告等進行市場趨勢分析運用AI工具協助企業執行市場行銷規劃和效益分析提出改進策略AI美術溝通師傳統職務多媒體動畫設計師、電腦繪圖人員AI化之後要在AI技術

In [16]:
import jieba
from collections import Counter
from wordcloud import WordCloud
from collections import Counter
import matplotlib.pyplot as plt


In [11]:
seg_list = jieba.cut(cleaned_text, cut_all=False)
words = list(seg_list)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ching\AppData\Local\Temp\jieba.cache
Loading model cost 0.591 seconds.
Prefix dict has been built successfully.


In [12]:
words


['輝達',
 'NVIDIA',
 '執行長',
 '黃',
 '仁勳',
 '、',
 '超微',
 'AMD',
 '執行長',
 '蘇姿丰',
 '相繼來',
 '台',
 '掀起',
 'AI',
 '熱潮',
 '！',
 '據',
 '104',
 '人力',
 '銀行',
 '徵才',
 '資料',
 '庫',
 '顯示',
 '目前',
 '市場',
 '上',
 '開出',
 '逾',
 '2.6',
 '萬個',
 'AI',
 '相關',
 '工作',
 '機會',
 '其中',
 '近',
 '六成',
 '集中',
 '於',
 '電子',
 '資訊',
 '、',
 '軟體',
 '、',
 '半導',
 '體業',
 '近',
 '兩成',
 '來',
 '自',
 '一般',
 '製',
 '造',
 '業',
 '而',
 '批',
 '發',
 '、',
 '零售',
 '、',
 '傳直',
 '銷業',
 '、',
 '法律',
 '、',
 '會計',
 '、',
 '顧問',
 '、',
 '研發及',
 '設計業',
 '也',
 '都',
 '有',
 '相關',
 '職缺',
 '釋出',
 '已見',
 '各產業',
 '同時',
 '需求',
 'AI',
 '人才',
 'AI',
 '應用',
 '人才',
 '搶手',
 '！',
 '跨',
 '五大',
 '產業',
 '企業',
 '徵才',
 '逾',
 '2.6',
 '萬人',
 '蘇姿丰',
 '日前',
 '接受',
 '媒體',
 '聯訪',
 '時',
 '指出',
 'AI',
 '目前',
 '仍',
 '在',
 '拓荒',
 '階段',
 '但',
 '她',
 '十分',
 '看好',
 '接下',
 '來',
 '5',
 '–',
 '10',
 '年',
 '發展',
 '超微',
 '會將',
 'AI',
 '當作',
 '首要',
 '投資',
 '項目',
 '其中',
 '硬體',
 '設備',
 '從',
 'CPU',
 '中央',
 '處',
 '理器',
 '、',
 'GPU',
 '繪圖',
 '處',
 '理器',
 '、',
 'FPGA',
 '現場',
 '可',
 '程式化',
 '邏輯'

In [13]:
# 計算兩個詞同時出現的次數

# 創建空的字典
co_occurrence = {}

# 遍歷兩個詞計算同時出現的情況
for i in range(len(words)):
    #第二個迴圈用來避免計算到兩個詞重複的情況
    for j in range(i+1, len(words)):
        #將同時出現的兩個詞建立一個key，並按造字母順序排序，確保後續無論哪個詞在先在後都能被計算
        key = tuple(sorted([words[i], words[j]]))
        #更新字典中該key值出現的次數，若字典並未存在該key值就創建一個新的並更新次數為1
        co_occurrence[key] = co_occurrence.get(key, 0) + 1

In [14]:
co_occurrence

{('NVIDIA', '輝達'): 1,
 ('執行長', '輝達'): 2,
 ('輝達', '黃'): 1,
 ('仁勳', '輝達'): 1,
 ('、', '輝達'): 31,
 ('超微', '輝達'): 2,
 ('AMD', '輝達'): 1,
 ('蘇姿丰', '輝達'): 2,
 ('相繼來', '輝達'): 1,
 ('台', '輝達'): 1,
 ('掀起', '輝達'): 1,
 ('AI', '輝達'): 48,
 ('熱潮', '輝達'): 1,
 ('輝達', '！'): 2,
 ('據', '輝達'): 1,
 ('104', '輝達'): 2,
 ('人力', '輝達'): 2,
 ('輝達', '銀行'): 2,
 ('徵才', '輝達'): 2,
 ('資料', '輝達'): 1,
 ('庫', '輝達'): 1,
 ('輝達', '顯示'): 1,
 ('目前', '輝達'): 4,
 ('市場', '輝達'): 5,
 ('上', '輝達'): 2,
 ('輝達', '開出'): 1,
 ('輝達', '逾'): 2,
 ('2.6', '輝達'): 2,
 ('萬個', '輝達'): 1,
 ('相關', '輝達'): 8,
 ('工作', '輝達'): 2,
 ('機會', '輝達'): 4,
 ('其中', '輝達'): 2,
 ('輝達', '近'): 2,
 ('六成', '輝達'): 1,
 ('輝達', '集中'): 1,
 ('於', '輝達'): 3,
 ('輝達', '電子'): 1,
 ('資訊', '輝達'): 2,
 ('軟體', '輝達'): 2,
 ('半導', '輝達'): 1,
 ('輝達', '體業'): 1,
 ('兩成', '輝達'): 1,
 ('來', '輝達'): 3,
 ('自', '輝達'): 1,
 ('一般', '輝達'): 1,
 ('製', '輝達'): 2,
 ('輝達', '造'): 1,
 ('業', '輝達'): 1,
 ('而', '輝達'): 1,
 ('批', '輝達'): 1,
 ('發', '輝達'): 1,
 ('輝達', '零售'): 1,
 ('傳直', '輝達'): 1,
 ('輝達', '銷業'): 1,
 ('法律', '輝達'): 1

In [17]:
# 計算每個詞出現的次數
word_freq = Counter(words)
print(word_freq)

Counter({'AI': 48, '、': 31, '和': 16, '與': 12, '應用': 10, '相關': 8, '職缺': 7, '技能': 7, '產品': 7, '企劃': 7, '技術': 7, '都': 6, '有': 6, '在': 6, '傳統': 6, '行銷': 6, '客戶': 6, '市場': 5, '方案': 5, '工具': 5, '目前': 4, '機會': 4, '媒體': 4, '發展': 4, '解決': 4, '對': 4, '能夠': 4, '團隊': 4, '分析': 4, '設計師': 4, '設計': 4, '於': 3, '來': 3, '也': 3, '人才': 3, '產業': 3, '企業': 3, '5': 3, '年': 3, '工程': 3, '工作者': 3, '要求': 3, '個': 3, '網站': 3, '合作': 3, '最佳': 3, '提供': 3, '協助': 3, '美術': 3, '專業': 3, '協調': 3, '執行長': 2, '超微': 2, '蘇姿丰': 2, '！': 2, '104': 2, '人力': 2, '銀行': 2, '徵才': 2, '上': 2, '逾': 2, '2.6': 2, '工作': 2, '其中': 2, '近': 2, '資訊': 2, '軟體': 2, '製': 2, '顧問': 2, '同時': 2, '需求': 2, '仍': 2, '拓荒': 2, '階段': 2, '但': 2, '她': 2, '十分': 2, '看好': 2, '接下': 2, '–': 2, '10': 2, '處': 2, '理器': 2, '繪圖': 2, '化': 2, '需要': 2, '研發': 2, '熱門': 2, '領域': 2, '進修': 2, '跳槽': 2, '轉職': 2, '或': 2, '原職': 2, '含金量': 2, '幫': 2, '自己': 2, '爭取': 2, '至少': 2, '一': 2, '成': 2, '以上': 2, '加薪': 2, '幅度': 2, '？': 2, '到': 2, '生成式': 2, 'ChatGPT': 2, 'Midjourney': 2, 'StableDiffusi

In [64]:
!pip install --upgrade wordcloud pillow

In [20]:
# font_path = './KAIU.TTF' # 標楷體
# wc = WordCloud(font_path=font_path, background_color="white", max_words=2000, width=800, height=600)
# wordcloud = wc.generate_from_frequencies(word_freq)

from wordcloud import WordCloud
from PIL import ImageFont

# 指定字体文件的路径
font_path = r'C:\Users\ching\kaiu.ttf'  # 替换为您的字体文件路径

# 创建一个Pillow字体对象
font = ImageFont.truetype(font_path, size=12)

# 创建WordCloud对象并设置字体
wc = WordCloud(font_path=font_path, background_color="white", max_words=2000, width=800, height=600)
wordcloud = wc.generate_from_frequencies(word_freq)


ValueError: Only supported for TrueType fonts

In [19]:
# 显示文字云
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')

NameError: name 'wordcloud' is not defined

<Figure size 720x576 with 0 Axes>